In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
catalog.list()

In [ ]:
articles = context.catalog.load('articles_sample')

In [ ]:
customers = context.catalog.load('customers_sample')

In [ ]:
transactions = context.catalog.load('transactions_sample')

### check results

In [ ]:
auto_articles = context.catalog.load('automated_articles_features')

In [ ]:
auto_customers = context.catalog.load('automated_customers_features')

In [ ]:
auto_articles.head()

In [ ]:
auto_articles.shape

In [ ]:
auto_customers.head()

In [ ]:
auto_customers.shape

In [ ]:
auto_articles[sorted(auto_articles.columns.to_list())]

In [ ]:
auto_customers[sorted(auto_customers.columns.to_list())].sort_values(by='COUNT(transactions)_all', ascending=False)

In [ ]:
from featuretools.selection import (
    # remove_highly_correlated_features() does not work with Booleans in current version of featuretools
    #remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)

### my custom correlation

In [ ]:
cor_matrix = auto_articles.corr().abs()

In [ ]:
np.fill_diagonal(cor_matrix.values, 0)

In [ ]:
cor_matrix

upper = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool_))

In [ ]:
def my_correlation(df):
    col_list = df.columns.to_list()
    correlated_columns = set()
    non_correlated_columns = set()
    for col in col_list:
        if col in correlated_columns:
            print('continue in a loop')
            continue
        non_correlated_columns.add(col)
        
        corr_cols_list = df.index[df.loc[:, col].ge(0.8)].to_list()
        if len(corr_cols_list)>0:
            correlated_columns |= set(corr_cols_list)
    print(f'{len(correlated_columns)=}')
    print(f'{len(non_correlated_columns)=}')
    return correlated_columns

In [ ]:
my_correlation(cor_matrix)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
auto_customers.isna().sum().reset_index().sort_values(by='index')

In [ ]:
auto_articles.isna().sum()

In [ ]:
pd.set_option('display.max_rows', 30)

In [ ]:
articles.shape, customers.shape, transactions.shape

In [ ]:
pd.set_option('display.max_columns', None)

### featuretools

In [ ]:
import featuretools as ft

In [ ]:
es = ft.EntitySet(id="kaggle_hm_data")

In [ ]:
import woodwork as ww

ww.list_logical_types()

In [ ]:
from woodwork.logical_types import Categorical, Boolean, AgeNullable, Double, NaturalLanguage

In [ ]:
customers.head()

In [ ]:
es = es.add_dataframe(
    dataframe_name="customers",
    dataframe=customers,
    index="customer_id",
    logical_types={
        "customer_id": Categorical,
        "FN": Boolean,
        "Active": Boolean,
        "club_member_status": Categorical,
        "fashion_news_frequency": Categorical,
        "age": AgeNullable
    },
)

In [ ]:
transactions['_index'] = transactions.index

In [ ]:
es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=transactions,
    index="_index",
    time_index="t_dat",
    logical_types={
        "customer_id": Categorical,
        "article_id": Categorical,
        "price": Double,
        "sales_channel_id": Categorical,
    },
)

In [ ]:
es = es.add_dataframe(
    dataframe_name="articles",
    dataframe=articles,
    index="article_id",
    logical_types={
        "article_id": Categorical,
        "detail_desc": NaturalLanguage,
    },
)

In [ ]:
es

In [ ]:
es = es.add_relationship("customers", "customer_id", "transactions", "customer_id")

In [ ]:
es = es.add_relationship("articles", "article_id", "transactions", "article_id")

In [ ]:
es

In [ ]:
es['articles'].ww.schema

In [ ]:
es['articles'].ww.schema

### articles

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
ft.primitives.list_primitives()

In [ ]:
pd.set_option('display.max_rows', 30)

In [ ]:
articles_feature_matrix, articles_feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="articles",
    agg_primitives=["sum", "mean", "median", "max", "min", "time_since_last", "count", "time_since_first"]
)

In [ ]:
list(enumerate(articles_feature_defs))

In [ ]:
articles_feature_defs[0].get_name()

In [ ]:
feats = [feature.get_name() for feature in articles_feature_defs if len(feature.base_features)>0]

In [ ]:
feats

In [ ]:
articles_feature_matrix[feats]

In [ ]:
articles_feature_defs[21].base_features

In [ ]:
articles_feature_defs[10].base_features

In [ ]:
dir(articles_feature_matrix)

In [ ]:
ft.describe_feature(articles_feature_defs[30])

In [ ]:
list(enumerate(articles_feature_defs))

### customers

In [ ]:
customers_feature_matrix, customers_feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    ignore_dataframes=["articles"],
    agg_primitives=["sum", "mean", "median", "max", "min", "time_since_last", "count", "time_since_first", "avg_time_between"],
)

In [ ]:
list(customers_feature_matrix.ww.select(exclude=[Boolean]).ww.schema.columns.keys())

In [ ]:
fm_to_check = customers_feature_matrix.ww.select(include=[Boolean])

In [ ]:
ft.primitives.list_primitives()

In [ ]:
customers_feature_matrix

In [ ]:
list(enumerate(customers_feature_defs))

### transactions

transactions features won't be used

In [ ]:
transactions_feature_matrix, transactions_feature_defs = ft.dfs(entityset=es, target_dataframe_name="transactions")

In [ ]:
transactions_feature_matrix.loc[0, :][0]

In [ ]:
transactions[transactions.customer_id=='0090852ad6a446ccf02a7d57ec7385c360ab33a813b870e7e725beae6a76ddd4']

In [ ]:
transactions_feature_matrix

In [ ]:
transactions_feature_defs

### remove

In [ ]:
customers_feature_matrix

In [ ]:
articles_feature_matrix

In [ ]:
from featuretools.selection import (
    remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)

In [ ]:
remove_highly_null_features(customers_feature_matrix, pct_null_threshold=0.2)

In [ ]:
new_fm, new_features = remove_single_value_features(customers_feature_matrix, features=customers_feature_defs)

In [ ]:
new_fm.shape

In [ ]:
set(new_features)-set(customers_feature_defs)

In [ ]:
customers_feature_matrix

bug in featuretools -> can't call .corr() on Boolean features

https://github.com/alteryx/featuretools/issues/2229

In [ ]:
new_fm, new_features = remove_highly_correlated_features(customers_feature_matrix, features=customers_feature_defs)

In [ ]:
cols = list(customers_feature_matrix.ww.select(exclude=[Boolean]).ww.schema.columns.keys())

In [ ]:
cols

In [ ]:
customers_feature_matrix.ww.select(exclude=[Boolean])

In [ ]:
new_fm, new_features = remove_highly_correlated_features(customers_feature_matrix.ww.select(exclude=[Boolean]),
                                                        features=cols)

In [ ]:
list(customers_feature_defs[2:])

In [ ]:
new_fm, new_features = remove_highly_correlated_features(customers_feature_matrix.iloc[:, 2:], features=customers_feature_defs[2:])

In [ ]:
new_fm